# Tutorial 2: Names and Binding

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-pure-passage-calculus/blob/main/notebooks/tutorial_02_names_and_binding.ipynb)

---

## The Problem of Names

In Year 726, a young scholar named Jorell Vance wrote to Kelleth Mund with a provocative question:

> *"Your passages use names—variables like x and y. But are names essential? Could we eliminate them entirely?"*

Mund was intrigued. He had always taken variable names as primitive. But Vance's question forced him to think carefully about what names actually *do* in passages.

In this tutorial, we explore **binding**—how names connect to values, and why the distinction between "free" and "bound" variables matters.

---

## Learning Objectives

By the end of this tutorial, you will:
1. Distinguish between **free** and **bound** variables
2. Understand **scope** and **shadowing**
3. Perform **alpha conversion** (renaming bound variables)
4. Identify when two passages are **alpha-equivalent**

## Setup

In [ ]:
import pandas as pd
import re

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions = pd.read_csv(BASE_URL + "passage_calculus_expressions.csv")
correspondence = pd.read_csv(BASE_URL + "mund_correspondence.csv")

print(f"Loaded {len(expressions)} expressions")
print(f"Loaded {len(correspondence)} letters")

## Part 1: Free vs. Bound Variables

Consider this passage:

```
λx.(x y)
```

There are two variables: `x` and `y`. But they have different roles:

- **`x` is bound**: The `λx` at the beginning "binds" this variable. When we apply the passage to an argument, `x` will be replaced.
- **`y` is free**: Nothing binds `y`. It refers to something outside the passage.

### Definition

A variable is **bound** if it appears under a λ (or `passage()`) with the same name.

A variable is **free** if it is not bound.

In [ ]:
# Let's look at expressions with free variables in Mund's catalog
with_free = expressions[expressions['free_variables'] != 'none']
with_free[['expression_id', 'lambda_notation', 'bound_variables', 'free_variables', 'notes']].head(10)

In [ ]:
# Count expressions by whether they have free variables
has_free = expressions['free_variables'] != 'none'
print(f"Expressions with free variables: {has_free.sum()}")
print(f"Closed expressions (no free variables): {(~has_free).sum()}")

### Why Free Variables Matter

Free variables represent **context**—information from outside the passage.

A **closed** expression (no free variables) is self-contained. It can be evaluated in isolation.

An **open** expression (has free variables) depends on its environment.

In [ ]:
# In Python, free variables come from the enclosing scope

y = 10  # Free in the lambda below
f = lambda x: x + y  # x is bound, y is free

print(f(5))  # Uses y=10 from outer scope

y = 100  # Change y
print(f(5))  # Same lambda, different result!

In [ ]:
# A closed expression doesn't depend on context
g = lambda x: lambda y: x + y  # Both x and y are bound

# This is self-contained—no external dependencies
print(g(3)(4))  # Always 7

## Part 2: Scope and Shadowing

When a passage contains nested λs with the same variable name, the inner one **shadows** the outer one.

```
λx.(λx.x)
```

Here the outer `x` is never used. The inner `λx` creates a new binding that shadows it.

In [ ]:
# Find shadowing examples in Mund's catalog
shadowing = expressions[expressions['lambda_notation'].str.contains(r'λx.*λx', regex=True)]
shadowing[['expression_name', 'lambda_notation', 'notes']]

In [ ]:
# Shadowing in Python
outer = lambda x: (lambda x: x)(99)
# The inner lambda ignores the outer x

print(outer("ignored"))  # Returns 99, not "ignored"

In [ ]:
# More complex shadowing
# λx.(λx.x x) - the inner x refers to the inner binding
complex_shadow = lambda x: (lambda x: x)(x)

# When applied to 5:
# - Outer x = 5
# - Inner lambda receives outer x (5) as its argument
# - Inner x = 5
# - Returns 5

print(complex_shadow(5))  # Returns 5

## Part 3: Alpha Conversion

Here's a key insight: **the names of bound variables don't matter**.

These passages do exactly the same thing:
- `λx.x`
- `λy.y`
- `λz.z`

They all return their input unchanged. The name is just a placeholder.

**Alpha conversion** is the process of renaming bound variables. Two passages that differ only by alpha conversion are called **alpha-equivalent**.

In [ ]:
# Find alpha variants in Mund's catalog
alpha_variants = expressions[expressions['expression_name'].str.contains('Alpha', na=False)]
alpha_variants[['expression_name', 'lambda_notation', 'notes']]

In [ ]:
# These are all alpha-equivalent
id_x = lambda x: x
id_y = lambda y: y  
id_z = lambda z: z

# They all do the same thing
print(id_x(5), id_y(5), id_z(5))  # All return 5

### When Alpha Conversion Matters

Alpha conversion becomes important when we need to avoid **variable capture**.

Consider substituting `y` for `x` in `λy.x`:
- Wrong: `λy.y` — the free `y` got captured by the binder!
- Right: First rename to `λz.x`, then substitute: `λz.y`

In [ ]:
# Variable capture example
# Suppose we have λy.x and want to substitute y for x

# WRONG approach: direct substitution gives λy.y
# This changed the meaning! λy.x returns its argument's outer context
# λy.y returns its argument itself

# RIGHT approach: alpha-convert first
# λy.x → λz.x (rename bound variable)
# Then substitute y for x: λz.y
# This preserves meaning: returns y from outer context

## Part 4: The Mund-Vance Correspondence

Let's explore the letters where Mund and Vance discussed these ideas.

In [ ]:
# Letters between Mund and Vance
vance_letters = correspondence[
    (correspondence['sender'].str.contains('Vance', na=False)) |
    (correspondence['recipient'].str.contains('Vance', na=False))
]
vance_letters[['date', 'sender', 'recipient', 'key_passage', 'significance']].head(10)

In [ ]:
# Find the letter where Vance first proposed eliminating names
naming_letters = correspondence[correspondence['primary_topic'].str.contains('nam', case=False, na=False)]
naming_letters[['date', 'sender', 'primary_topic', 'key_passage', 'significance']]

In [ ]:
# Breakthrough letters about names
breakthroughs = correspondence[
    (correspondence['significance'] == 'breakthrough') &
    (correspondence['sender'].isin(['Jorell Vance', 'Kelleth Mund']))
]
breakthroughs[['date', 'sender', 'key_passage', 'notes']]

## Part 5: Implementing Free Variable Detection

Let's build a simple function to find free variables in a passage.

In [ ]:
def free_variables(expr):
    """
    Find free variables in a lambda expression string.
    
    This is a simplified parser for expressions like:
    - λx.x
    - λx.λy.(x y)
    - (λx.x y)
    """
    expr = expr.strip()
    
    # Base case: single variable
    if re.match(r'^[a-z]$', expr):
        return {expr}
    
    # Lambda abstraction: λx.body
    match = re.match(r'^λ([a-z])\.(.+)$', expr)
    if match:
        var = match.group(1)
        body = match.group(2)
        # Free in body, minus the bound variable
        return free_variables(body) - {var}
    
    # Application: (expr1 expr2)
    if expr.startswith('(') and expr.endswith(')'):
        inner = expr[1:-1]
        # Simple split (doesn't handle nested parens well)
        parts = inner.split(' ', 1)
        if len(parts) == 2:
            return free_variables(parts[0]) | free_variables(parts[1])
    
    return set()

# Test
print("λx.x:", free_variables("λx.x"))
print("λx.y:", free_variables("λx.y"))
print("(x y):", free_variables("(x y)"))

In [ ]:
# Check our function against the catalog
sample = expressions[expressions['free_variables'] != 'none'].head(5)

for _, row in sample.iterrows():
    expr = row['lambda_notation']
    catalog_free = set(row['free_variables'].split(';')) if pd.notna(row['free_variables']) else set()
    computed_free = free_variables(expr)
    print(f"{expr}")
    print(f"  Catalog: {row['free_variables']}")
    print(f"  Computed: {computed_free}")
    print()

## Exercises

### Exercise 1: Identify Free and Bound Variables

For each expression, list the free and bound variables:

1. `λx.(x y z)`
2. `λx.λy.(x y)`
3. `λf.λx.(f (f x))`
4. `(λx.x)(λy.y)`

In [ ]:
# Exercise 1 workspace

# 1. λx.(x y z)
# Bound: ?
# Free: ?

# 2. λx.λy.(x y)
# Bound: ?
# Free: ?

# 3. λf.λx.(f (f x))
# Bound: ?
# Free: ?

# 4. (λx.x)(λy.y)
# Bound: ?
# Free: ?

### Exercise 2: Alpha Equivalence

Which pairs are alpha-equivalent?

1. `λx.x` and `λy.y`
2. `λx.λy.x` and `λa.λb.a`
3. `λx.y` and `λy.y`
4. `λx.λy.x` and `λy.λx.y`

In [ ]:
# Exercise 2 workspace

# Test by implementing and comparing behavior

# 1. λx.x vs λy.y
e1a = lambda x: x
e1b = lambda y: y
# Are they equivalent?

# 2. λx.λy.x vs λa.λb.a
e2a = lambda x: lambda y: x
e2b = lambda a: lambda b: a
# Are they equivalent?

# 3. λx.y vs λy.y - careful here!
# The first has a FREE y, the second has a BOUND y

# 4. λx.λy.x vs λy.λx.y
e4a = lambda x: lambda y: x
e4b = lambda y: lambda x: y
# Test: e4a(1)(2) vs e4b(1)(2)

### Exercise 3: Avoiding Capture

When substituting `y` for `x` in each expression, identify if alpha conversion is needed:

1. `λz.x` (substitute y for x)
2. `λy.x` (substitute y for x)
3. `λy.(x y)` (substitute y for x)

In [ ]:
# Exercise 3 workspace

# 1. λz.x [y/x] = ?
# Is y captured by λz? 

# 2. λy.x [y/x] = ?
# Is y captured by λy?

# 3. λy.(x y) [y/x] = ?
# Is y captured by λy?

## Summary

In this tutorial, we learned:

1. **Bound variables** appear under a λ with the same name; they are placeholders
2. **Free variables** refer to the external context
3. **Shadowing** occurs when an inner λ uses the same name as an outer one
4. **Alpha conversion** renames bound variables without changing meaning
5. **Alpha-equivalent** expressions differ only in the names of bound variables
6. **Variable capture** is a danger when substituting; alpha conversion prevents it

### Next Tutorial

In Tutorial 3, we'll explore **application and substitution**—the core operation of the passage calculus: beta reduction.